In [1]:
import urllib2
import json
import pandas as pd
import csv

Upload posts CSV file into a pandas dataframe. 
CSV contains 3 columns, illustrated below.
Note ascii problems.

In [3]:
posts_df = pd.read_csv('top-blog-posts.csv')
posts_df.head()

,Page Title,Page,Unique Page Views
0,Check a vehicle�۪s MOT status online | Matters...,mattersoftesting.blog.gov.uk/check-a-vehicles-...,"338,547"
1,DVLA digital services,dvladigital.blog.gov.uk/,"134,249"
2,Renew your tax disc online | Moving On,movingon.blog.gov.uk/renew-your-tax-disc-online/,"117,345"
3,Reminder ��� HGV speed limit changes in Englan...,movingon.blog.gov.uk/reminder-hgv-speed-limit-...,"107,211"
4,It now costs less to retain your personalised ...,dvladigital.blog.gov.uk/2015/03/09/it-now-cost...,"75,147"


Add the 'https://' onto every URL slug in the posts_df['Page'] column

In [4]:
posts_df['Page'] = 'https://' + posts_df['Page'].astype(str)
posts_df.head()

,Page Title,Page,Unique Page Views
0,Check a vehicle�۪s MOT status online | Matters...,https://mattersoftesting.blog.gov.uk/check-a-v...,"338,547"
1,DVLA digital services,https://dvladigital.blog.gov.uk/,"134,249"
2,Renew your tax disc online | Moving On,https://movingon.blog.gov.uk/renew-your-tax-di...,"117,345"
3,Reminder ��� HGV speed limit changes in Englan...,https://movingon.blog.gov.uk/reminder-hgv-spee...,"107,211"
4,It now costs less to retain your personalised ...,https://dvladigital.blog.gov.uk/2015/03/09/it-...,"75,147"


Define a function that calls up data from Sharedcount.com for each URL and uses my API key

In [5]:
def get_social_metrics(url, api_key):
    sharedcount_response = urllib2.urlopen('https://free.sharedcount.com/?url=' + url + '&apikey=' + api_key)
    return json.load(sharedcount_response)

API key defined.
Runs function against each row in that column, placing data into a new column.

In [6]:
SHAREDCOUNT_API_KEY = '2c8787d463e1be55a6ef68b1a341ccc0ccb41697'
posts_df['sharedcount_metrics'] = map(lambda Page: get_social_metrics(Page, SHAREDCOUNT_API_KEY), posts_df['Page'])

In [63]:
posts_df.head()

,Page Title,Page,Unique Page Views,sharedcount_metrics
0,Check a vehicle�۪s MOT status online | Matters...,https://mattersoftesting.blog.gov.uk/check-a-v...,"338,547","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'..."
1,DVLA digital services,https://dvladigital.blog.gov.uk/,"134,249","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'..."
2,Renew your tax disc online | Moving On,https://movingon.blog.gov.uk/renew-your-tax-di...,"117,345","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'..."
3,Reminder ��� HGV speed limit changes in Englan...,https://movingon.blog.gov.uk/reminder-hgv-spee...,"107,211","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'..."
4,It now costs less to retain your personalised ...,https://dvladigital.blog.gov.uk/2015/03/09/it-...,"75,147","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'..."


Extract selected fields from the sharedcount.com data and put them in their own columns

In [64]:
posts_df['Twitter'] = map(lambda sharedcount: sharedcount['Twitter'], posts_df['sharedcount_metrics'])
posts_df['LinkedIn'] = map(lambda sharedcount: sharedcount['LinkedIn'], posts_df['sharedcount_metrics'])
posts_df['Facebook shares'] = map(lambda sharedcount: sharedcount['Facebook']['share_count'], posts_df['sharedcount_metrics'])
posts_df['Facebook likes'] = map(lambda sharedcount: sharedcount['Facebook']['like_count'], posts_df['sharedcount_metrics'])
posts_df['Facebook comments'] = map(lambda sharedcount: sharedcount['Facebook']['comment_count'], posts_df['sharedcount_metrics'])

In [65]:
posts_df.head()

,Page Title,Page,Unique Page Views,sharedcount_metrics,Twitter,LinkedIn,Facebook shares,Facebook likes,Facebook comments
0,Check a vehicle�۪s MOT status online | Matters...,https://mattersoftesting.blog.gov.uk/check-a-v...,"338,547","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",21,3,20,9,4
1,DVLA digital services,https://dvladigital.blog.gov.uk/,"134,249","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",42,20,0,0,0
2,Renew your tax disc online | Moving On,https://movingon.blog.gov.uk/renew-your-tax-di...,"117,345","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",3,0,10,3,12
3,Reminder ��� HGV speed limit changes in Englan...,https://movingon.blog.gov.uk/reminder-hgv-spee...,"107,211","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",238,243,928,3545,3682
4,It now costs less to retain your personalised ...,https://dvladigital.blog.gov.uk/2015/03/09/it-...,"75,147","{u'StumbleUpon': 0, u'Reddit': 0, u'Delicious'...",47,9,110,120,75


Writes dataframe to CSV

In [66]:
posts_df.to_csv('top-blog-posts_social-shares.csv')